In [ ]:
© Liu Xiaoquan, Assistant Professor, HKCHC

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import r2_score
from itertools import combinations
import warnings
import os
import json
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
warnings.filterwarnings('ignore')

data = pd.read_csv('/Users/xiaoquanliu/Desktop/Book_DataCode1/第七章/DL_Data16_processed.csv')
print(f"Original data shape: {data.shape}")

numeric_cols = data.columns[2:] 
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())
original_feature_names = list(data.columns[3:36]) 

original_features = data.columns[3:36]  
interaction_features = []
interaction_feature_names = []

for i, j in combinations(range(len(original_features)), 2):
    col_name = f'interaction_{i}_{j}'
    interaction_name = f'{original_features[i]}_×_{original_features[j]}'
    data[col_name] = data[original_features[i]] * data[original_features[j]]
    interaction_features.append(col_name)
    interaction_feature_names.append(interaction_name)

print(f"Generated {len(interaction_features)} interaction features")

all_feature_names = original_feature_names + interaction_feature_names

all_features = list(original_features) + interaction_features
X = data[all_features].values
y = data['Return'].values
stock_ids = data.iloc[:, 0].values
time_ids = data.iloc[:, 1].values

X_original = data[original_features].values

train_idx = int(len(data) * 0.8)
X_train, X_test = X[:train_idx], X[train_idx:]
y_train, y_test = y[:train_idx], y[train_idx:]
time_train, time_test = time_ids[:train_idx], time_ids[train_idx:]
stock_train, stock_test = stock_ids[:train_idx], stock_ids[train_idx:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class DualStructureNN(keras.Model):
    def __init__(self, num_factors=10, num_features=None,
                 factor_loading_layers=[256, 128], 
                 factor_extraction_layers=[256, 128]):
        super(DualStructureNN, self).__init__()
        self.num_factors = num_factors
        self.num_features = num_features
        
        # 因子载荷网络 g(z_i,t; θ)
        self.factor_loading_layers = []
        for i, units in enumerate(factor_loading_layers):
            self.factor_loading_layers.append(
                layers.Dense(units, activation='relu', name=f'g_layer_{i}')
            )
        self.factor_loading_output = layers.Dense(
            num_factors, activation=None, name='g_output'
        )
        
        # 因子提取网络 h(x_t+1; φ)
        self.factor_extraction_layers = []
        for i, units in enumerate(factor_extraction_layers):
            self.factor_extraction_layers.append(
                layers.Dense(units, activation='relu', name=f'h_layer_{i}')
            )
        self.factor_extraction_output = layers.Dense(
            num_factors, activation=None, name='h_output'
        )
        
    def compute_factor_loadings(self, z):
        """计算因子载荷 β_i,t = g(z_i,t; θ)"""
        x = z
        for layer in self.factor_loading_layers:
            x = layer(x)
        return self.factor_loading_output(x)
    
    def compute_factors(self, x_weighted):
        """计算因子 f_t+1 = h(x_t+1; φ)"""
        if len(x_weighted.shape) == 1:
            x_weighted = tf.expand_dims(x_weighted, 0)
        
        x = x_weighted
        for layer in self.factor_extraction_layers:
            x = layer(x)
        factors = self.factor_extraction_output(x)
        
        if factors.shape[0] == 1:
            factors = tf.squeeze(factors, axis=0)
            
        return factors
    
    def call(self, inputs, training=None):
        z_batch, x_weighted = inputs

        factor_loadings = self.compute_factor_loadings(z_batch)

        factors = self.compute_factors(x_weighted)

        if len(factors.shape) == 1:
            factors = tf.expand_dims(factors, 0)
            factors = tf.tile(factors, [tf.shape(factor_loadings)[0], 1])
        
        predictions = tf.reduce_sum(factor_loadings * factors, axis=1)
        
        return predictions, factor_loadings, factors

def prepare_time_based_data(X, y, time_ids):
    """按时间组织数据"""
    unique_times = np.unique(time_ids)
    time_data = {}
    
    for t in unique_times:
        mask = time_ids == t
        time_data[t] = {
            'X': X[mask],
            'y': y[mask],
            'indices': np.where(mask)[0]
        }
    
    return time_data, unique_times


print("\n" + "="*60)
print("功能1: 测试不同因子数量的双结构神经网络R平方")
print("="*60)

factor_numbers = [29, 30, 31]
nn_results = {}

for num_factors in factor_numbers:
    print(f"\n训练因子数量为 {num_factors} 的模型...")

    model = DualStructureNN(
        num_factors=num_factors, 
        num_features=X_train_scaled.shape[1],
        factor_loading_layers=[256, 128, 64],
        factor_extraction_layers=[256, 128, 64]
    )
    
    optimizer = keras.optimizers.Adam(learning_rate=0.001)

    train_time_data, unique_times_train = prepare_time_based_data(
        X_train_scaled, y_train, time_train
    )

    @tf.function
    def train_step_time(z_batch, y_batch, x_weighted):
        with tf.GradientTape() as tape:
            predictions, _, _ = model([z_batch, x_weighted], training=True)
            loss = tf.reduce_mean(tf.square(y_batch - predictions))
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    epochs = 300
    for epoch in range(epochs):
        epoch_losses = []
        
        for t in unique_times_train:
            t_data = train_time_data[t]
            if len(t_data['X']) < 2:
                continue
                
            x_weighted = np.dot(t_data['X'].T, t_data['y']) / len(t_data['y'])
            
            loss = train_step_time(
                tf.constant(t_data['X'], dtype=tf.float32),
                tf.constant(t_data['y'], dtype=tf.float32),
                tf.constant(x_weighted, dtype=tf.float32)
            )
            epoch_losses.append(loss.numpy())
        
        if (epoch + 1) % 50 == 0:
            print(f"  Epoch {epoch + 1}/{epochs}, Loss: {np.mean(epoch_losses):.6f}")
    
    X_full_scaled = np.vstack([X_train_scaled, X_test_scaled])
    y_full = np.concatenate([y_train, y_test])
    time_full = np.concatenate([time_train, time_test])
    
    all_factors_full = []
    full_factor_indices = []
    
    full_time_data, unique_times_full = prepare_time_based_data(
        X_full_scaled, y_full, time_full
    )
    
    for t in unique_times_full:
        t_data = full_time_data[t]
        if len(t_data['X']) < 2:
            continue
        
        x_weighted = np.dot(t_data['X'].T, t_data['y']) / len(t_data['y'])
        
        _, _, factors = model(
            [tf.constant(t_data['X'], dtype=tf.float32),
             tf.constant(x_weighted, dtype=tf.float32)],
            training=False
        )
        
        factors_np = factors.numpy()
        if len(factors_np.shape) == 1:
            factors_expanded = np.tile(factors_np, (len(t_data['X']), 1))
        else:
            factors_expanded = factors_np
        
        all_factors_full.extend(factors_expanded)
        full_factor_indices.extend(t_data['indices'])
    
    all_factors_full = np.array(all_factors_full)

    X_factors_full = all_factors_full
    y_returns_full = y_full[full_factor_indices]
    
    linear_reg = LinearRegression()
    linear_reg.fit(X_factors_full, y_returns_full)
    y_pred_full = linear_reg.predict(X_factors_full)
    r2_full = r2_score(y_returns_full, y_pred_full)

    n = len(y_returns_full)
    p = num_factors
    adj_r2_full = 1 - (1 - r2_full) * (n - 1) / (n - p - 1)
    
    nn_results[num_factors] = {
        'r2': r2_full,
        'adj_r2': adj_r2_full,
        'model': model,
        'factors': all_factors_full,
        'linear_reg': linear_reg,
        'y_returns': y_returns_full,
        'indices': full_factor_indices
    }
    
    print(f"  因子数量 {num_factors}: R² = {r2_full:.4f}, Adjusted R² = {adj_r2_full:.4f}")

print("\n" + "="*60)
print("功能2: 使用statsmodels进行30个因子线性回归分析")
print("="*60)

model_30 = nn_results[30]
X_factors_30 = model_30['factors']
y_returns_30 = model_30['y_returns']

factor_names = [f'Factor_{i+1}' for i in range(30)]

factors_df = pd.DataFrame(X_factors_30, columns=factor_names)
factors_df['Return'] = y_returns_30

X_sm = factors_df[factor_names]
y_sm = factors_df['Return']
X_sm_with_const = sm.add_constant(X_sm)

ols_model = sm.OLS(y_sm, X_sm_with_const).fit()

print("\n=== OLS回归结果摘要 ===")
print(ols_model.summary())

regression_stats = {
    'model_summary': {
        'r_squared': float(ols_model.rsquared),
        'adj_r_squared': float(ols_model.rsquared_adj),
        'f_statistic': float(ols_model.fvalue),
        'f_pvalue': float(ols_model.f_pvalue),
        'aic': float(ols_model.aic),
        'bic': float(ols_model.bic),
        'log_likelihood': float(ols_model.llf),
        'n_observations': int(ols_model.nobs),
        'df_residuals': int(ols_model.df_resid),
        'df_model': int(ols_model.df_model)
    },
    'coefficients': {}
}

for i, param in enumerate(ols_model.params.index):
    regression_stats['coefficients'][param] = {
        'coefficient': float(ols_model.params[param]),
        'std_error': float(ols_model.bse[param]),
        't_value': float(ols_model.tvalues[param]),
        'p_value': float(ols_model.pvalues[param]),
        'conf_int_lower': float(ols_model.conf_int().iloc[i, 0]),
        'conf_int_upper': float(ols_model.conf_int().iloc[i, 1])
    }

def get_significance_stars(p_value):
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    elif p_value < 0.1:
        return '.'
    else:
        return ''

print("\n=== 系数详细信息表 ===")
print(f"{'变量':<12} {'系数':>12} {'标准误差':>12} {'t值':>10} {'p值':>10} {'显著性':>8} {'95%置信区间':>25}")
print("-" * 95)

factor_results = []
for param in ols_model.params.index:
    coef = ols_model.params[param]
    std_err = ols_model.bse[param]
    t_val = ols_model.tvalues[param]
    p_val = ols_model.pvalues[param]
    stars = get_significance_stars(p_val)
    conf_lower = ols_model.conf_int().loc[param, 0]
    conf_upper = ols_model.conf_int().loc[param, 1]
    
    print(f"{param:<12} {coef:>12.6f} {std_err:>12.6f} {t_val:>10.3f} {p_val:>10.4f} {stars:>8} [{conf_lower:>8.4f}, {conf_upper:>8.4f}]")
    
    if param != 'const': 
        factor_results.append({
            'factor': param,
            'coefficient': float(coef),
            'std_error': float(std_err),
            't_value': float(t_val),
            'p_value': float(p_val),
            'significance': stars,
            'conf_int_lower': float(conf_lower),
            'conf_int_upper': float(conf_upper)
        })



# 显著性分析
print(f"\n=== 显著性分析 ===")
significant_factors = [f for f in factor_results if f['p_value'] < 0.05]
highly_significant_factors = [f for f in factor_results if f['p_value'] < 0.01]

print(f"显著因子数量 (p < 0.05): {len(significant_factors)}/10")
print(f"高度显著因子数量 (p < 0.01): {len(highly_significant_factors)}/10")

if significant_factors:
    print(f"\n显著因子列表:")
    for factor in significant_factors:
        print(f"  {factor['factor']}: 系数={factor['coefficient']:.6f}, p值={factor['p_value']:.4f}{factor['significance']}")


print("\n" + "="*60)
print("功能3: 对所有显著因子进行原始特征和交互特征分析")
print("="*60)
significant_factors_for_analysis = []
for result in factor_results:
    if result['p_value'] < 0.05:  
        significant_factors_for_analysis.append({
            'index': int(result['factor'].split('_')[1]) - 1, 
            'factor': result['factor'],
            'coefficient': result['coefficient'],
            'p_value': result['p_value'],
            'significance': result['significance']
        })


all_significant_results = {}

if significant_factors_for_analysis:
    print(f"\n找到 {len(significant_factors_for_analysis)} 个显著因子，开始逐个分析...")
 
    X_all_subset = X[model_30['indices']]  
    print(f"使用特征数量: {X_all_subset.shape[1]} (原始特征: {len(original_feature_names)}, 交互特征: {len(interaction_feature_names)})")
    scaler_all = StandardScaler()
    X_all_scaled = scaler_all.fit_transform(X_all_subset)

    for i, factor_info in enumerate(significant_factors_for_analysis):
        factor_index = factor_info['index']
        factor_name = factor_info['factor']
        
        print(f"\n{'='*50}")
        print(f"分析因子 {i+1}/{len(significant_factors_for_analysis)}: {factor_name}")
        print(f"系数: {factor_info['coefficient']:.6f}")
        print(f"p值: {factor_info['p_value']:.4f}{factor_info['significance']}")
        print(f"{'='*50}")

        y_factor = X_factors_30[:, factor_index]

        print(f"对因子 {factor_name} 进行Lasso回归...")
        alphas = np.logspace(-4, 1, 50)
        best_alpha = None
        best_score = -np.inf

        for alpha in alphas:
            lasso = Lasso(alpha=alpha, max_iter=2000, random_state=42)
            lasso.fit(X_all_scaled, y_factor)
            score = lasso.score(X_all_scaled, y_factor)
            if score > best_score:
                best_score = score
                best_alpha = alpha

        lasso_final = Lasso(alpha=best_alpha, max_iter=2000, random_state=42)
        lasso_final.fit(X_all_scaled, y_factor)

        feature_importance = np.abs(lasso_final.coef_)

        non_zero_indices = np.where(lasso_final.coef_ != 0)[0]
        non_zero_importance = feature_importance[non_zero_indices]
        non_zero_sorted_indices = non_zero_indices[np.argsort(non_zero_importance)[::-1]]
        
        print(f"Lasso回归结果 (alpha={best_alpha:.4f}):")
        print(f"R² score: {best_score:.4f}")
        print(f"非零系数特征数量: {len(non_zero_indices)}/{len(all_feature_names)}")

        original_features_selected = []
        interaction_features_selected = []
        
        for idx in non_zero_sorted_indices:
            coef = lasso_final.coef_[idx]
            abs_coef = np.abs(coef)
            feature_name = all_feature_names[idx]
            
            feature_info = {
                'feature_name': feature_name,
                'coefficient': float(coef),
                'abs_coefficient': float(abs_coef),
                'feature_index': int(idx)
            }
            
            if idx < len(original_feature_names):
                original_features_selected.append(feature_info)
            else:
                interaction_features_selected.append(feature_info)

        print(f"\n显著的原始特征 ({len(original_features_selected)} 个):")
        if original_features_selected:
            print(f"{'排名':<6} {'特征名称':<30} {'Lasso系数':>12} {'绝对值':>12}")
            print("-" * 65)
            for rank, feature in enumerate(original_features_selected[:10], 1):
                print(f"{rank:<6} {feature['feature_name']:<30} {feature['coefficient']:>12.6f} {feature['abs_coefficient']:>12.6f}")
        else:
            print("  无显著的原始特征")

        print(f"\n显著的交互特征 ({len(interaction_features_selected)} 个):")
        if interaction_features_selected:
            print(f"{'排名':<6} {'特征名称':<40} {'Lasso系数':>12} {'绝对值':>12}")
            print("-" * 75)
            for rank, feature in enumerate(interaction_features_selected[:10], 1):
                print(f"{rank:<6} {feature['feature_name']:<40} {feature['coefficient']:>12.6f} {feature['abs_coefficient']:>12.6f}")
        else:
            print("  无显著的交互特征")
        total_original = len(original_features_selected)
        total_interaction = len(interaction_features_selected)
        original_pct = (total_original / len(original_feature_names)) * 100 if len(original_feature_names) > 0 else 0
        interaction_pct = (total_interaction / len(interaction_feature_names)) * 100 if len(interaction_feature_names) > 0 else 0
        
        print(f"\n特征选择统计:")
        print(f"  原始特征: {total_original}/{len(original_feature_names)} ({original_pct:.1f}%)")
        print(f"  交互特征: {total_interaction}/{len(interaction_feature_names)} ({interaction_pct:.1f}%)")
        print(f"  总计: {total_original + total_interaction}/{len(all_feature_names)}")

        factor_result = {
            'factor_info': factor_info,
            'lasso_alpha': float(best_alpha),
            'lasso_r2': float(best_score),
            'total_features_selected': len(non_zero_indices),
            'original_features': {
                'count': total_original,
                'percentage': float(original_pct),
                'features': original_features_selected
            },
            'interaction_features': {
                'count': total_interaction,
                'percentage': float(interaction_pct),
                'features': interaction_features_selected
            }
        }
        
        all_significant_results[factor_name] = factor_result

else:
    print("\n没有找到显著的因子 (p < 0.05)")


class IPCA:
    def __init__(self, n_factors=5):
        self.n_factors = n_factors
        self.gamma = None
        
    def fit(self, X, y, time_ids):
        n_features = X.shape[1]
        self.gamma = np.random.randn(n_features, self.n_factors) * 0.01
        
        unique_times = np.unique(time_ids)
        
        for iteration in range(50):
            # E步：估计因子
            factors = {}
            
            for t in unique_times:
                t_mask = time_ids == t
                X_t = X[t_mask]
                y_t = y[t_mask]
                
                if len(X_t) > 0:
                    beta_t = X_t @ self.gamma
                                    
                    try:
                        f_t = np.linalg.solve(beta_t.T @ beta_t, beta_t.T @ y_t)
                    except:
                        f_t = np.linalg.lstsq(beta_t, y_t, rcond=None)[0]
                    factors[t] = f_t
            
            # M步：更新Γ
            numerator = np.zeros((n_features, self.n_factors))
            denominator = np.zeros((self.n_factors, self.n_factors))
            
            for t in unique_times:
                if t not in factors:
                    continue
                    
                t_mask = time_ids == t
                X_t = X[t_mask]
                y_t = y[t_mask]
                f_t = factors[t]
                
                numerator += X_t.T @ y_t[:, np.newaxis] @ f_t[np.newaxis, :]
                denominator += np.outer(f_t, f_t) * len(X_t)
            
            try:
                self.gamma = numerator @ np.linalg.inv(denominator)
            except:
                self.gamma = numerator @ np.linalg.pinv(denominator)
            
            if iteration % 10 == 0:
                loss = 0
                for t in unique_times:
                    if t not in factors:
                        continue
                    t_mask = time_ids == t
                    X_t = X[t_mask]
                    y_t = y[t_mask]
                    beta_t = X_t @ self.gamma
                    pred_t = beta_t @ factors[t]
                    loss += np.sum((y_t - pred_t)**2)
                print(f"IPCA Iteration {iteration}, Loss: {loss:.4f}")
    
    def predict(self, X_test, y_test, time_test):
        unique_times_test = np.unique(time_test)
        predictions = np.zeros(len(X_test))
        
        for t in unique_times_test:
            t_mask = time_test == t
            X_t = X_test[t_mask]
            y_t = y_test[t_mask]
            
            if len(X_t) > 0:
                beta_t = X_t @ self.gamma
                try:
                    f_t = np.linalg.solve(beta_t.T @ beta_t, beta_t.T @ y_t)
                except:
                    f_t = np.linalg.lstsq(beta_t, y_t, rcond=None)[0]
                
                predictions[t_mask] = beta_t @ f_t
        
        return predictions

print("\n" + "="*60)
print("训练IPCA模型进行对比...")
print("="*60)
ipca = IPCA(n_factors=3)
ipca.fit(X_train_scaled, y_train, time_train)

y_pred_ipca = ipca.predict(X_test_scaled, y_test, time_test)
r2_ipca = 1 - np.sum((y_test - y_pred_ipca)**2) / np.sum((y_test - np.mean(y_test))**2)

print("\n" + "="*60)
print("最终结果汇总")
print("="*60)

print("\n双结构神经网络不同因子数量的R²对比:")
for num_factors in factor_numbers:
    print(f"因子数量 {num_factors}: R² = {nn_results[num_factors]['r2']:.4f}, Adjusted R² = {nn_results[num_factors]['adj_r2']:.4f}")

print(f"\nIPCA模型 R²: {r2_ipca:.4f}")

save_dir = '/Users/xiaoquanliu/Desktop/Book_DataCode1/第七章/factors_data_30_1'
os.makedirs(save_dir, exist_ok=True)

all_results = {
    'nn_results': {
        str(k): {
            'r2': float(v['r2']),
            'adj_r2': float(v['adj_r2'])
        } for k, v in nn_results.items()
    },
    'statsmodels_regression': regression_stats,
    'factor_significance': factor_results,
    'ipca_r2': float(r2_ipca)
}


if all_significant_results:
    all_results['significant_factors_analysis'] = {
        'total_significant_factors': len(significant_factors_for_analysis),
        'results_by_factor': all_significant_results
    }
    
    summary_stats = {
        'total_factors_analyzed': len(significant_factors_for_analysis),
        'avg_original_features_selected': np.mean([r['original_features']['count'] for r in all_significant_results.values()]),
        'avg_interaction_features_selected': np.mean([r['interaction_features']['count'] for r in all_significant_results.values()]),
        'avg_original_percentage': np.mean([r['original_features']['percentage'] for r in all_significant_results.values()]),
        'avg_interaction_percentage': np.mean([r['interaction_features']['percentage'] for r in all_significant_results.values()]),
        'avg_lasso_r2': np.mean([r['lasso_r2'] for r in all_significant_results.values()])
    }
    
    all_results['significant_factors_analysis']['summary_statistics'] = summary_stats

    summary_df = pd.DataFrame([
        {
            'Factor': factor_name,
            'Original_Coefficient': result['factor_info']['coefficient'],
            'P_Value': result['factor_info']['p_value'],
            'Significance': result['factor_info']['significance'],
            'Lasso_R2': result['lasso_r2'],
            'Original_Features_Count': result['original_features']['count'],
            'Interaction_Features_Count': result['interaction_features']['count'],
            'Total_Features_Selected': result['total_features_selected'],
            'Original_Features_Pct': result['original_features']['percentage'],
            'Interaction_Features_Pct': result['interaction_features']['percentage']
        }
        for factor_name, result in all_significant_results.items()
    ])
    
    summary_df.to_csv(os.path.join(save_dir, 'significant_factors_summary.csv'), index=False)

    for factor_name, result in all_significant_results.items():
        if result['total_features_selected'] > 0:
            all_selected_features = []

            for feature in result['original_features']['features']:
                all_selected_features.append({
                    'Feature_Type': '原始',
                    'Feature_Name': feature['feature_name'],
                    'Coefficient': feature['coefficient'],
                    'Abs_Coefficient': feature['abs_coefficient'],
                    'Feature_Index': feature['feature_index']
                })

            for feature in result['interaction_features']['features']:
                all_selected_features.append({
                    'Feature_Type': '交互',
                    'Feature_Name': feature['feature_name'],
                    'Coefficient': feature['coefficient'],
                    'Abs_Coefficient': feature['abs_coefficient'],
                    'Feature_Index': feature['feature_index']
                })

            all_selected_features.sort(key=lambda x: x['Abs_Coefficient'], reverse=True)
            
            factor_features_df = pd.DataFrame(all_selected_features)
            factor_csv_filename = f'factor_{factor_name}_selected_features.csv'
            factor_features_df.to_csv(os.path.join(save_dir, factor_csv_filename), index=False)
    
    print(f"\n" + "="*60)
    print("所有显著因子分析汇总")
    print("="*60)
    print(f"分析的显著因子数量: {len(significant_factors_for_analysis)}")
    print(f"平均选择的原始特征数量: {summary_stats['avg_original_features_selected']:.1f}")
    print(f"平均选择的交互特征数量: {summary_stats['avg_interaction_features_selected']:.1f}")
    print(f"平均原始特征选择比例: {summary_stats['avg_original_percentage']:.1f}%")
    print(f"平均交互特征选择比例: {summary_stats['avg_interaction_percentage']:.1f}%")
    print(f"平均Lasso R²: {summary_stats['avg_lasso_r2']:.4f}")
    
    print(f"\n汇总结果已保存至: significant_factors_summary.csv")
    print(f"各因子详细特征已分别保存至: factor_[因子名]_selected_features.csv")

with open(os.path.join(save_dir, 'enhanced_results_with_interaction_features.json'), 'w') as f:
    json.dump(all_results, f, indent=2, ensure_ascii=False)

full_factors_df = pd.DataFrame(nn_results[30]['factors'], 
                               columns=[f'Factor_{i+1}' for i in range(30)])
full_factors_df['Return'] = nn_results[30]['y_returns']
full_factors_df.to_csv(os.path.join(save_dir, 'factors_30_with_interaction.csv'), index=False)

feature_mapping = {
    'original_features': {
        'count': len(original_feature_names),
        'names': original_feature_names
    },
    'interaction_features': {
        'count': len(interaction_feature_names),
        'names': interaction_feature_names
    },
    'all_features': {
        'count': len(all_feature_names),
        'names': all_feature_names
    }
}

with open(os.path.join(save_dir, 'feature_mapping.json'), 'w') as f:
    json.dump(feature_mapping, f, indent=2, ensure_ascii=False)

with open(os.path.join(save_dir, 'statsmodels_regression_summary.txt'), 'w') as f:
    f.write("=== Statsmodels OLS回归完整结果 ===\n\n")
    f.write(str(ols_model.summary()))
    f.write("\n\n=== 模型诊断检验结果 ===\n")

    if 'diagnostics' in regression_stats:
        diagnostics = regression_stats['diagnostics']
        
        if 'breusch_pagan_stat' in diagnostics:
            f.write(f"\nBreusch-Pagan异方差检验:\n")
            f.write(f"  统计量: {diagnostics['breusch_pagan_stat']:.4f}\n")
            f.write(f"  p值: {diagnostics['breusch_pagan_pvalue']:.4f}\n")
            if diagnostics['breusch_pagan_pvalue'] < 0.05:
                f.write("  结论: 存在异方差 (p < 0.05)\n")
            else:
                f.write("  结论: 不存在异方差 (p >= 0.05)\n")
        
        if 'durbin_watson' in diagnostics:
            f.write(f"\nDurbin-Watson自相关检验:\n")
            f.write(f"  统计量: {diagnostics['durbin_watson']:.4f}\n")
            if diagnostics['durbin_watson'] < 1.5:
                f.write("  结论: 可能存在正自相关\n")
            elif diagnostics['durbin_watson'] > 2.5:
                f.write("  结论: 可能存在负自相关\n")
            else:
                f.write("  结论: 无明显自相关\n")
        
        if 'jarque_bera_stat' in diagnostics:
            f.write(f"\nJarque-Bera正态性检验:\n")
            f.write(f"  统计量: {diagnostics['jarque_bera_stat']:.4f}\n")
            f.write(f"  p值: {diagnostics['jarque_bera_pvalue']:.4f}\n")
            f.write(f"  偏度: {diagnostics['skewness']:.4f}\n")
            f.write(f"  峰度: {diagnostics['kurtosis']:.4f}\n")
            if diagnostics['jarque_bera_pvalue'] < 0.05:
                f.write("  结论: 残差不服从正态分布 (p < 0.05)\n")
            else:
                f.write("  结论: 残差服从正态分布 (p >= 0.05)\n")

coefficients_df = pd.DataFrame([
    {
        'Variable': param,
        'Coefficient': regression_stats['coefficients'][param]['coefficient'],
        'Std_Error': regression_stats['coefficients'][param]['std_error'],
        'T_Value': regression_stats['coefficients'][param]['t_value'],
        'P_Value': regression_stats['coefficients'][param]['p_value'],
        'Conf_Int_Lower': regression_stats['coefficients'][param]['conf_int_lower'],
        'Conf_Int_Upper': regression_stats['coefficients'][param]['conf_int_upper'],
        'Significance': get_significance_stars(regression_stats['coefficients'][param]['p_value'])
    }
    for param in ols_model.params.index
])
coefficients_df.to_csv(os.path.join(save_dir, 'regression_coefficients_detailed.csv'), index=False)

print(f"\n" + "="*80)
print("优化后的功能3分析总结")
print("="*80)

print(f"\n特征统计:")
print(f"  原始特征数量: {len(original_feature_names)}")
print(f"  交互特征数量: {len(interaction_feature_names)}")
print(f"  总特征数量: {len(all_feature_names)}")

if all_significant_results:
    print(f"\n显著因子分析结果:")
    print(f"  分析的显著因子数量: {len(significant_factors_for_analysis)}")
    
    if 'summary_statistics' in all_results.get('significant_factors_analysis', {}):
        summary_stats = all_results['significant_factors_analysis']['summary_statistics']
        print(f"  平均选择的原始特征数量: {summary_stats['avg_original_features_selected']:.1f}")
        print(f"  平均选择的交互特征数量: {summary_stats['avg_interaction_features_selected']:.1f}")
        print(f"  平均原始特征选择比例: {summary_stats['avg_original_percentage']:.1f}%")
        print(f"  平均交互特征选择比例: {summary_stats['avg_interaction_percentage']:.1f}%")
        print(f"  平均Lasso R²: {summary_stats['avg_lasso_r2']:.4f}")

print(f"\n模型性能对比:")
for num_factors in factor_numbers:
    print(f"  神经网络({num_factors}因子): R² = {nn_results[num_factors]['r2']:.4f}")
print(f"  IPCA模型: R² = {r2_ipca:.4f}")

print(f"\n所有结果已保存至目录: {save_dir}")


2025-06-17 21:03:40.934342: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Original data shape: (901767, 36)
Generated 528 interaction features

功能1: 测试不同因子数量的双结构神经网络R平方

训练因子数量为 29 的模型...


2025-06-17 21:03:56.396199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


  Epoch 50/300, Loss: 0.666516
  Epoch 100/300, Loss: 0.551360
  Epoch 150/300, Loss: 0.508458
  Epoch 200/300, Loss: 0.480846
  Epoch 250/300, Loss: 0.469531
  Epoch 300/300, Loss: 0.456188
  因子数量 29: R² = 0.2030, Adjusted R² = 0.2030

训练因子数量为 30 的模型...
  Epoch 50/300, Loss: 0.634350
  Epoch 100/300, Loss: 0.541658
  Epoch 150/300, Loss: 0.502937
  Epoch 200/300, Loss: 0.485460
  Epoch 250/300, Loss: 0.465482
  Epoch 300/300, Loss: 0.454704
  因子数量 30: R² = 0.2010, Adjusted R² = 0.2010

训练因子数量为 31 的模型...
  Epoch 50/300, Loss: 0.649730
  Epoch 100/300, Loss: 0.558351
  Epoch 150/300, Loss: 0.517213
  Epoch 200/300, Loss: 0.492337
  Epoch 250/300, Loss: 0.478330
  Epoch 300/300, Loss: 0.461948
  因子数量 31: R² = 0.2029, Adjusted R² = 0.2028

功能2: 使用statsmodels进行30个因子线性回归分析

=== OLS回归结果摘要 ===
                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.201
Model:                            OLS   Adj.

In [2]:

print("\n" + "="*60)
print("时间序列交叉验证（3 fold）- 30因子模型稳健性验证")
print("数据分4份：训练集递增，测试集为下一个1/4")
print("="*60)

data_cv = pd.read_csv('/Users/xiaoquanliu/Desktop/Book_DataCode1/第七章/DL_Data16_processed.csv')
data_cv[numeric_cols] = data_cv[numeric_cols].fillna(data_cv[numeric_cols].mean())

for i, j in combinations(range(len(original_features)), 2):
  col_name = f'interaction_{i}_{j}'
  data_cv[col_name] = data_cv[original_features[i]] * data_cv[original_features[j]]

X_cv = data_cv[all_features].values
y_cv = data_cv['Return'].values
time_cv = data_cv.iloc[:, 1].values

unique_times_cv = np.unique(time_cv)
n_times = len(unique_times_cv)
quarter_size = n_times // 4 

print(f"总时间点数: {n_times}")
print(f"每个1/4包含时间点数: {quarter_size}")
print(f"时间范围: {unique_times_cv[0]} 到 {unique_times_cv[-1]}")

cv_results = []
cv_detailed_results = []

for fold in range(3):
  print(f"\n--- 第 {fold+1} 折验证 ---")
  train_end_idx = quarter_size * (fold + 1)
  test_start_idx = train_end_idx             
  test_end_idx = quarter_size * (fold + 2)   
  if fold == 2:
      test_end_idx = n_times
  
  train_times = unique_times_cv[:train_end_idx]
  test_times = unique_times_cv[test_start_idx:test_end_idx]
  
  print(f"训练时间范围: {train_times[0]} 到 {train_times[-1]} (共{len(train_times)}个时间点)")
  print(f"测试时间范围: {test_times[0]} 到 {test_times[-1]} (共{len(test_times)}个时间点)")
  print(f"训练集占比: {len(train_times)/n_times:.1%}, 测试集占比: {len(test_times)/n_times:.1%}")

  assert len(np.intersect1d(train_times, test_times)) == 0, f"第{fold+1}折: 训练集和测试集时间重叠!"
  assert len(train_times) > 0 and len(test_times) > 0, f"第{fold+1}折: 训练集或测试集为空!"
  assert train_times[-1] < test_times[0], f"第{fold+1}折: 训练集时间应该早于测试集时间!"

  train_mask = np.isin(time_cv, train_times)
  test_mask = np.isin(time_cv, test_times)
  
  X_train_cv, X_test_cv = X_cv[train_mask], X_cv[test_mask]
  y_train_cv, y_test_cv = y_cv[train_mask], y_cv[test_mask]
  time_train_cv, time_test_cv = time_cv[train_mask], time_cv[test_mask]
  
  print(f"训练样本数: {np.sum(train_mask)}, 测试样本数: {np.sum(test_mask)}")

  if np.sum(train_mask) < 100:
      print(f"警告: 第{fold+1}折训练样本数过少 ({np.sum(train_mask)})")
  if np.sum(test_mask) < 50:
      print(f"警告: 第{fold+1}折测试样本数过少 ({np.sum(test_mask)})")

  scaler_cv = StandardScaler()
  X_train_scaled_cv = scaler_cv.fit_transform(X_train_cv)
  X_test_scaled_cv = scaler_cv.transform(X_test_cv)

  model_cv = DualStructureNN(num_factors=30, num_features=X_train_scaled_cv.shape[1])
  optimizer_cv = keras.optimizers.Adam(learning_rate=0.001)

  train_time_data_cv, unique_times_train_cv = prepare_time_based_data(X_train_scaled_cv, y_train_cv, time_train_cv)

  training_losses = []
  best_loss = float('inf')
  patience_counter = 0
  patience = 10
  
  for epoch in range(200):
      epoch_losses = []
      for t in unique_times_train_cv:
          t_data = train_time_data_cv[t]
          if len(t_data['X']) < 2: continue
          x_weighted = np.dot(t_data['X'].T, t_data['y']) / len(t_data['y'])
          with tf.GradientTape() as tape:
              predictions, _, _ = model_cv([tf.constant(t_data['X'], dtype=tf.float32), tf.constant(x_weighted, dtype=tf.float32)], training=True)
              loss = tf.reduce_mean(tf.square(tf.constant(t_data['y'], dtype=tf.float32) - predictions))
          gradients = tape.gradient(loss, model_cv.trainable_variables)
          optimizer_cv.apply_gradients(zip(gradients, model_cv.trainable_variables))
          epoch_losses.append(loss.numpy())
      
      if epoch_losses:
          current_loss = np.mean(epoch_losses)
          training_losses.append(current_loss)

          if current_loss < best_loss:
              best_loss = current_loss
              patience_counter = 0
          else:
              patience_counter += 1
              
          if patience_counter >= patience:
              print(f"早停于第{epoch+1}轮，最佳损失: {best_loss:.6f}")
              break

  test_time_data_cv, unique_times_test_cv = prepare_time_based_data(X_test_scaled_cv, y_test_cv, time_test_cv)
  all_factors_cv = []
  test_indices_cv = []
  
  for t in unique_times_test_cv:
      t_data = test_time_data_cv[t]
      if len(t_data['X']) < 2: continue
      x_weighted = np.dot(t_data['X'].T, t_data['y']) / len(t_data['y'])
      _, _, factors = model_cv([tf.constant(t_data['X'], dtype=tf.float32), tf.constant(x_weighted, dtype=tf.float32)], training=False)
      factors_np = factors.numpy()
      if len(factors_np.shape) == 1:
          factors_expanded = np.tile(factors_np, (len(t_data['X']), 1))
      else:
          factors_expanded = factors_np
      all_factors_cv.extend(factors_expanded)
      test_indices_cv.extend(t_data['indices'])
  
  all_factors_cv = np.array(all_factors_cv)
  y_test_subset_cv = y_test_cv[test_indices_cv]

  linear_reg_cv = LinearRegression()
  linear_reg_cv.fit(all_factors_cv, y_test_subset_cv)
  y_pred_cv = linear_reg_cv.predict(all_factors_cv)
  r2_cv = r2_score(y_test_subset_cv, y_pred_cv)

  n_cv = len(y_test_subset_cv)
  adj_r2_cv = 1 - (1 - r2_cv) * (n_cv - 1) / (n_cv - 31)
  mse_cv = np.mean((y_test_subset_cv - y_pred_cv)**2)
  mae_cv = np.mean(np.abs(y_test_subset_cv - y_pred_cv))
  final_loss = training_losses[-1] if training_losses else np.nan

  residuals = y_test_subset_cv - y_pred_cv
  tracking_error = np.std(residuals) * np.sqrt(252)  
  info_ratio = (np.mean(residuals) * 252) / tracking_error if tracking_error > 0 else 0

  fold_result = {
      'Fold': fold + 1,
      'Train_Time_Start': str(train_times[0]),
      'Train_Time_End': str(train_times[-1]),
      'Test_Time_Start': str(test_times[0]),
      'Test_Time_End': str(test_times[-1]),
      'Train_Samples': int(np.sum(train_mask)),
      'Test_Samples': int(n_cv),
      'Train_Time_Periods': len(train_times),
      'Test_Time_Periods': len(test_times),
      'Train_Ratio': len(train_times)/n_times,
      'Test_Ratio': len(test_times)/n_times,
      'R_Squared': r2_cv,
      'Adjusted_R_Squared': adj_r2_cv,
      'MSE': mse_cv,
      'MAE': mae_cv,
      'Information_Ratio': info_ratio,
      'Tracking_Error': tracking_error,
      'Final_Training_Loss': final_loss,
      'Training_Epochs': len(training_losses),
      'Factors_Generated': all_factors_cv.shape[1]
  }
  
  cv_results.append(fold_result)
  cv_detailed_results.append({
      'fold': fold+1, 
      'factors': all_factors_cv, 
      'predictions': y_pred_cv, 
      'actual': y_test_subset_cv,
      'coefficients': linear_reg_cv.coef_,
      'residuals': residuals
  })
  
  print(f"第{fold+1}折结果:")
  print(f"  R²: {r2_cv:.4f}, 调整R²: {adj_r2_cv:.4f}")
  print(f"  MSE: {mse_cv:.6f}, MAE: {mae_cv:.6f}")
  print(f"  信息比率: {info_ratio:.4f}, 跟踪误差: {tracking_error:.4f}")
  print(f"  训练轮数: {len(training_losses)}")

cv_results_df = pd.DataFrame(cv_results)

summary_stats = {
  'Metric': ['R_Squared', 'Adjusted_R_Squared', 'MSE', 'MAE', 'Information_Ratio'],
  'Mean': [
      cv_results_df['R_Squared'].mean(),
      cv_results_df['Adjusted_R_Squared'].mean(),
      cv_results_df['MSE'].mean(),
      cv_results_df['MAE'].mean(),
      cv_results_df['Information_Ratio'].mean()
  ],
  'Std': [
      cv_results_df['R_Squared'].std(),
      cv_results_df['Adjusted_R_Squared'].std(),
      cv_results_df['MSE'].std(),
      cv_results_df['MAE'].std(),
      cv_results_df['Information_Ratio'].std()
  ],
  'Min': [
      cv_results_df['R_Squared'].min(),
      cv_results_df['Adjusted_R_Squared'].min(),
      cv_results_df['MSE'].min(),
      cv_results_df['MAE'].min(),
      cv_results_df['Information_Ratio'].min()
  ],
  'Max': [
      cv_results_df['R_Squared'].max(),
      cv_results_df['Adjusted_R_Squared'].max(),
      cv_results_df['MSE'].max(),
      cv_results_df['MAE'].max(),
      cv_results_df['Information_Ratio'].max()
  ]
}

summary_df = pd.DataFrame(summary_stats)
coefficients_summary = []
for i, result in enumerate(cv_detailed_results):
  for j, coef in enumerate(result['coefficients']):
      coefficients_summary.append({
          'Fold': i + 1,
          'Factor': f'Factor_{j+1}',
          'Coefficient': coef
      })

coefficients_df = pd.DataFrame(coefficients_summary)
coefficients_pivot = coefficients_df.pivot(index='Factor', columns='Fold', values='Coefficient')
coefficients_pivot['Mean'] = coefficients_pivot.mean(axis=1)
coefficients_pivot['Std'] = coefficients_pivot.std(axis=1)
coefficients_pivot['CV'] = coefficients_pivot['Std'] / np.abs(coefficients_pivot['Mean'])  # 变异系数
cv_results_df.to_csv(os.path.join(save_dir, 'cross_validation_results.csv'), index=False)
summary_df.to_csv(os.path.join(save_dir, 'cross_validation_summary.csv'), index=False)
coefficients_pivot.to_csv(os.path.join(save_dir, 'cross_validation_coefficients.csv'))

for i, result in enumerate(cv_detailed_results):
  fold_predictions_df = pd.DataFrame({
      'Actual_Return': result['actual'],
      'Predicted_Return': result['predictions'],
      'Residual': result['residuals'],
      'Abs_Residual': np.abs(result['residuals'])
  })
  fold_predictions_df.to_csv(os.path.join(save_dir, f'fold_{i+1}_predictions.csv'), index=False)

print(f"\n{'='*60}")
print(f"=== 交叉验证结果汇总 ===")
print(f"{'='*60}")
print(f"R² - 均值: {cv_results_df['R_Squared'].mean():.4f} ± {cv_results_df['R_Squared'].std():.4f}")
print(f"调整R² - 均值: {cv_results_df['Adjusted_R_Squared'].mean():.4f} ± {cv_results_df['Adjusted_R_Squared'].std():.4f}")
print(f"MSE - 均值: {cv_results_df['MSE'].mean():.6f} ± {cv_results_df['MSE'].std():.6f}")
print(f"信息比率 - 均值: {cv_results_df['Information_Ratio'].mean():.4f} ± {cv_results_df['Information_Ratio'].std():.4f}")

r2_cv_coef = cv_results_df['R_Squared'].std() / cv_results_df['R_Squared'].mean()
print(f"\n=== 稳健性评估 ===")
print(f"R²变异系数: {r2_cv_coef:.4f}")
print(f"R²稳健性评估: {'稳健' if r2_cv_coef < 0.1 else '中等' if r2_cv_coef < 0.2 else '不够稳健'}")

print(f"\n=== 性能趋势分析 ===")
for i in range(3):
  print(f"第{i+1}折 (训练集占{cv_results_df.iloc[i]['Train_Ratio']:.1%}): R² = {cv_results_df.iloc[i]['R_Squared']:.4f}")

print(f"\n=== 保存的CSV文件 ===")
print(f"1. 交叉验证主要结果: cross_validation_results.csv")
print(f"2. 统计汇总: cross_validation_summary.csv") 
print(f"3. 因子系数汇总: cross_validation_coefficients.csv")
print(f"4. 各折预测详情: fold_1_predictions.csv, fold_2_predictions.csv, fold_3_predictions.csv")


时间序列交叉验证（3 fold）- 30因子模型稳健性验证
数据分4份：训练集递增，测试集为下一个1/4
总时间点数: 482
每个1/4包含时间点数: 120
时间范围: 1/10/22 到 9/9/22

--- 第 1 折验证 ---
训练时间范围: 1/10/22 到 12/15/21 (共120个时间点)
测试时间范围: 12/15/22 到 4/10/23 (共120个时间点)
训练集占比: 24.9%, 测试集占比: 24.9%
训练样本数: 224183, 测试样本数: 225217
早停于第143轮，最佳损失: 0.423661
第1折结果:
  R²: 0.1722, 调整R²: 0.1721
  MSE: 0.780472, MAE: 0.593374
  信息比率: 0.0001, 跟踪误差: 14.0242
  训练轮数: 143

--- 第 2 折验证 ---
训练时间范围: 1/10/22 到 4/10/23 (共240个时间点)
测试时间范围: 4/11/22 到 7/11/23 (共120个时间点)
训练集占比: 49.8%, 测试集占比: 24.9%
训练样本数: 449400, 测试样本数: 224261
早停于第171轮，最佳损失: 0.427456
第2折结果:
  R²: 0.1933, 调整R²: 0.1932
  MSE: 0.932847, MAE: 0.663227
  信息比率: -0.0070, 跟踪误差: 15.3322
  训练轮数: 171

--- 第 3 折验证 ---
训练时间范围: 1/10/22 到 7/11/23 (共360个时间点)
测试时间范围: 7/12/22 到 9/9/22 (共122个时间点)
训练集占比: 74.7%, 测试集占比: 25.3%
训练样本数: 673661, 测试样本数: 228106
早停于第127轮，最佳损失: 0.516831
第3折结果:
  R²: 0.1814, 调整R²: 0.1813
  MSE: 0.739761, MAE: 0.581417
  信息比率: -0.0001, 跟踪误差: 13.6536
  训练轮数: 127

=== 交叉验证结果汇总 ===
R² - 均值: 0.1823 ± 0.0105
调整R² - 均值: 0.18

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from itertools import combinations
import warnings
import os
import json
warnings.filterwarnings('ignore')

data = pd.read_csv('/Users/xiaoquanliu/Desktop/Book_DataCode1/第七章/DL_Data16_processed.csv')
print(f"Original data shape: {data.shape}")

numeric_cols = data.columns[2:] 
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

original_feature_names = list(data.columns[3:36]) 
original_features = data.columns[3:36]  
interaction_features = []
interaction_feature_names = []

for i, j in combinations(range(len(original_features)), 2):
    col_name = f'interaction_{i}_{j}'
    interaction_name = f'{original_features[i]}_×_{original_features[j]}'
    data[col_name] = data[original_features[i]] * data[original_features[j]]
    interaction_features.append(col_name)
    interaction_feature_names.append(interaction_name)

print(f"Generated {len(interaction_features)} interaction features")

all_feature_names = original_feature_names + interaction_feature_names

all_features = list(original_features) + interaction_features
X = data[all_features].values
y = data['Return'].values
stock_ids = data.iloc[:, 0].values
time_ids = data.iloc[:, 1].values

train_idx = int(len(data) * 0.8)
X_train, X_test = X[:train_idx], X[train_idx:]
y_train, y_test = y[:train_idx], y[train_idx:]
time_train, time_test = time_ids[:train_idx], time_ids[train_idx:]
stock_train, stock_test = stock_ids[:train_idx], stock_ids[train_idx:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class DualStructureNN(keras.Model):
    def __init__(self, num_factors=10, num_features=None,
                 factor_loading_layers=[256, 128], 
                 factor_extraction_layers=[256, 128]):
        super(DualStructureNN, self).__init__()
        self.num_factors = num_factors
        self.num_features = num_features
        
        self.factor_loading_layers = []
        for i, units in enumerate(factor_loading_layers):
            self.factor_loading_layers.append(
                layers.Dense(units, activation='relu', name=f'g_layer_{i}')
            )
        self.factor_loading_output = layers.Dense(
            num_factors, activation=None, name='g_output'
        )
        
        self.factor_extraction_layers = []
        for i, units in enumerate(factor_extraction_layers):
            self.factor_extraction_layers.append(
                layers.Dense(units, activation='relu', name=f'h_layer_{i}')
            )
        self.factor_extraction_output = layers.Dense(
            num_factors, activation=None, name='h_output'
        )
        
    def compute_factor_loadings(self, z):
        """计算因子载荷 β_i,t = g(z_i,t; θ)"""
        x = z
        for layer in self.factor_loading_layers:
            x = layer(x)
        return self.factor_loading_output(x)
    
    def compute_factors(self, x_weighted):
        """计算因子 f_t+1 = h(x_t+1; φ)"""
        if len(x_weighted.shape) == 1:
            x_weighted = tf.expand_dims(x_weighted, 0)
        
        x = x_weighted
        for layer in self.factor_extraction_layers:
            x = layer(x)
        factors = self.factor_extraction_output(x)
        
        if factors.shape[0] == 1:
            factors = tf.squeeze(factors, axis=0)
            
        return factors
    
    def call(self, inputs, training=None):
        z_batch, x_weighted = inputs
        factor_loadings = self.compute_factor_loadings(z_batch)
        factors = self.compute_factors(x_weighted)
        if len(factors.shape) == 1:
            factors = tf.expand_dims(factors, 0)
            factors = tf.tile(factors, [tf.shape(factor_loadings)[0], 1])
        
        predictions = tf.reduce_sum(factor_loadings * factors, axis=1)
        
        return predictions, factor_loadings, factors
    
def prepare_time_based_data(X, y, time_ids):
    """按时间组织数据"""
    unique_times = np.unique(time_ids)
    time_data = {}
    
    for t in unique_times:
        mask = time_ids == t
        time_data[t] = {
            'X': X[mask],
            'y': y[mask],
            'indices': np.where(mask)[0]
        }
    
    return time_data, unique_times

print("\n" + "="*60)
print("扩展功能1: 测试25-35个因子的双结构神经网络R平方")
print("="*60)

factor_numbers = list(range(25, 36)) 
nn_results = {}

for num_factors in factor_numbers:
    print(f"\n训练因子数量为 {num_factors} 的模型...")

    model = DualStructureNN(
        num_factors=num_factors, 
        num_features=X_train_scaled.shape[1],
        factor_loading_layers=[256, 128, 64],
        factor_extraction_layers=[256, 128, 64]
    )
    
    optimizer = keras.optimizers.Adam(learning_rate=0.001)

    train_time_data, unique_times_train = prepare_time_based_data(
        X_train_scaled, y_train, time_train
    )
    
    @tf.function
    def train_step_time(z_batch, y_batch, x_weighted):
        with tf.GradientTape() as tape:
            predictions, _, _ = model([z_batch, x_weighted], training=True)
            loss = tf.reduce_mean(tf.square(y_batch - predictions))
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss
 
    epochs = 300
    for epoch in range(epochs):
        epoch_losses = []
        
        for t in unique_times_train:
            t_data = train_time_data[t]
            if len(t_data['X']) < 2:
                continue
                
            x_weighted = np.dot(t_data['X'].T, t_data['y']) / len(t_data['y'])
            
            loss = train_step_time(
                tf.constant(t_data['X'], dtype=tf.float32),
                tf.constant(t_data['y'], dtype=tf.float32),
                tf.constant(x_weighted, dtype=tf.float32)
            )
            epoch_losses.append(loss.numpy())
        
        if (epoch + 1) % 50 == 0:
            print(f"  Epoch {epoch + 1}/{epochs}, Loss: {np.mean(epoch_losses):.6f}")

    X_full_scaled = np.vstack([X_train_scaled, X_test_scaled])
    y_full = np.concatenate([y_train, y_test])
    time_full = np.concatenate([time_train, time_test])
    
    all_factors_full = []
    full_factor_indices = []
    
    full_time_data, unique_times_full = prepare_time_based_data(
        X_full_scaled, y_full, time_full
    )
    
    for t in unique_times_full:
        t_data = full_time_data[t]
        if len(t_data['X']) < 2:
            continue
        
        x_weighted = np.dot(t_data['X'].T, t_data['y']) / len(t_data['y'])
        
        _, _, factors = model(
            [tf.constant(t_data['X'], dtype=tf.float32),
             tf.constant(x_weighted, dtype=tf.float32)],
            training=False
        )
        
        factors_np = factors.numpy()
        if len(factors_np.shape) == 1:
            factors_expanded = np.tile(factors_np, (len(t_data['X']), 1))
        else:
            factors_expanded = factors_np
        
        all_factors_full.extend(factors_expanded)
        full_factor_indices.extend(t_data['indices'])
    
    all_factors_full = np.array(all_factors_full)

    X_factors_full = all_factors_full
    y_returns_full = y_full[full_factor_indices]
    
    linear_reg = LinearRegression()
    linear_reg.fit(X_factors_full, y_returns_full)
    y_pred_full = linear_reg.predict(X_factors_full)
    r2_full = r2_score(y_returns_full, y_pred_full)
 
    n = len(y_returns_full)
    p = num_factors
    adj_r2_full = 1 - (1 - r2_full) * (n - 1) / (n - p - 1)
    
    nn_results[num_factors] = {
        'r2': r2_full,
        'adj_r2': adj_r2_full,
        'n_observations': n,
        'n_factors': p
    }
    
    print(f"  因子数量 {num_factors}: R² = {r2_full:.4f}, Adjusted R² = {adj_r2_full:.4f}")

print("\n" + "="*60)
print("25-35个因子的R²统计结果汇总")
print("="*60)

results_table = []
for num_factors in factor_numbers:
    result = nn_results[num_factors]
    results_table.append({
        'Factors': num_factors,
        'R_squared': result['r2'],
        'Adj_R_squared': result['adj_r2'],
        'N_observations': result['n_observations']
    })

results_df = pd.DataFrame(results_table)
print("\n详细结果表:")
print(f"{'因子数量':<8} {'R²':>12} {'调整R²':>12} {'观测数':>10}")
print("-" * 50)
for _, row in results_df.iterrows():
    print(f"{row['Factors']:<8} {row['R_squared']:>12.6f} {row['Adj_R_squared']:>12.6f} {row['N_observations']:>10.0f}")

best_r2_idx = results_df['R_squared'].idxmax()
best_adj_r2_idx = results_df['Adj_R_squared'].idxmax()

print(f"\n性能分析:")
print(f"最高R²: {results_df.loc[best_r2_idx, 'Factors']}个因子 (R² = {results_df.loc[best_r2_idx, 'R_squared']:.6f})")
print(f"最高调整R²: {results_df.loc[best_adj_r2_idx, 'Factors']}个因子 (调整R² = {results_df.loc[best_adj_r2_idx, 'Adj_R_squared']:.6f})")

r2_values = results_df['R_squared'].values
adj_r2_values = results_df['Adj_R_squared'].values

print(f"\nR²统计:")
print(f"  平均值: {np.mean(r2_values):.6f}")
print(f"  标准差: {np.std(r2_values):.6f}")
print(f"  最小值: {np.min(r2_values):.6f}")
print(f"  最大值: {np.max(r2_values):.6f}")

print(f"\n调整R²统计:")
print(f"  平均值: {np.mean(adj_r2_values):.6f}")
print(f"  标准差: {np.std(adj_r2_values):.6f}")
print(f"  最小值: {np.min(adj_r2_values):.6f}")
print(f"  最大值: {np.max(adj_r2_values):.6f}")

save_dir = '/Users/xiaoquanliu/Desktop/Book_DataCode1/第七章/factors_25_35_results'
os.makedirs(save_dir, exist_ok=True)

results_df.to_csv(os.path.join(save_dir, 'factors_25_35_r_squared_results.csv'), index=False)

complete_results = {
    'experiment_info': {
        'factor_range': [25, 35],
        'total_models_trained': len(factor_numbers),
        'training_epochs': 300,
        'train_test_split': 0.8
    },
    'results_summary': {
        'best_r2_factors': int(results_df.loc[best_r2_idx, 'Factors']),
        'best_r2_value': float(results_df.loc[best_r2_idx, 'R_squared']),
        'best_adj_r2_factors': int(results_df.loc[best_adj_r2_idx, 'Factors']),
        'best_adj_r2_value': float(results_df.loc[best_adj_r2_idx, 'Adj_R_squared']),
        'r2_statistics': {
            'mean': float(np.mean(r2_values)),
            'std': float(np.std(r2_values)),
            'min': float(np.min(r2_values)),
            'max': float(np.max(r2_values))
        },
        'adj_r2_statistics': {
            'mean': float(np.mean(adj_r2_values)),
            'std': float(np.std(adj_r2_values)),
            'min': float(np.min(adj_r2_values)),
            'max': float(np.max(adj_r2_values))
        }
    },
    'detailed_results': {
        str(k): {
            'r2': float(v['r2']),
            'adj_r2': float(v['adj_r2']),
            'n_observations': int(v['n_observations']),
            'n_factors': int(v['n_factors'])
        } for k, v in nn_results.items()
    }
}

with open(os.path.join(save_dir, 'factors_25_35_complete_results.json'), 'w') as f:
    json.dump(complete_results, f, indent=2, ensure_ascii=False)

print(f"\n" + "="*60)
print("实验完成")
print("="*60)
print(f"结果已保存至目录: {save_dir}")
print(f"- CSV文件: factors_25_35_r_squared_results.csv")
print(f"- JSON文件: factors_25_35_complete_results.json")
print(f"\n共训练了 {len(factor_numbers)} 个模型，因子数量范围: {min(factor_numbers)}-{max(factor_numbers)}")


Original data shape: (901767, 36)
Generated 528 interaction features

扩展功能1: 测试25-35个因子的双结构神经网络R平方

训练因子数量为 25 的模型...
  Epoch 50/300, Loss: 0.643653
  Epoch 100/300, Loss: 0.553692
  Epoch 150/300, Loss: 0.520674
  Epoch 200/300, Loss: 0.489024
  Epoch 250/300, Loss: 0.473093
  Epoch 300/300, Loss: 0.465716
  因子数量 25: R² = 0.2026, Adjusted R² = 0.2026

训练因子数量为 26 的模型...
  Epoch 50/300, Loss: 0.641906
  Epoch 100/300, Loss: 0.553808
  Epoch 150/300, Loss: 0.513077
  Epoch 200/300, Loss: 0.491131
  Epoch 250/300, Loss: 0.471027
  Epoch 300/300, Loss: 0.463428
  因子数量 26: R² = 0.2019, Adjusted R² = 0.2019

训练因子数量为 27 的模型...
  Epoch 50/300, Loss: 0.646159
  Epoch 100/300, Loss: 0.564456
  Epoch 150/300, Loss: 0.521822
  Epoch 200/300, Loss: 0.509633
  Epoch 250/300, Loss: 0.481696
  Epoch 300/300, Loss: 0.465961
  因子数量 27: R² = 0.2024, Adjusted R² = 0.2024

训练因子数量为 28 的模型...
  Epoch 50/300, Loss: 0.637951
  Epoch 100/300, Loss: 0.554683
  Epoch 150/300, Loss: 0.513736
  Epoch 200/300, Loss: